# SMS Spam classification with spaCy

SpaCy is an open-source library for Natural Language Processing in Python. Giving a word/text as a parameter to spaCy's nlp model results in a doc object which holds its tokens and vectors. Word Embeddings (vectors) are used for calculating similarity between given words/texts. For example words like "Sea", "Ocean", "Water" will have similiar vector values compared to words like "Train" or "Notebook". Another example is substracting vector of a word "Queen" from a vector of a word "King" and then adding the result to a word "Man" will result in a word "Woman". Those vectors can be used as features for machine learning models and then predicting the sentiment/class of a text.

Run the following cell to complete data and english language spaCy model load. 

In [ ]:
import pandas as pd
import spacy
import numpy as np

data= pd.read_csv('../input/spam-text-message-classification/SPAM text message 20170820 - Data.csv')

nlp = spacy.load("en_core_web_lg")

# disabling other pipes that aren't needed
with nlp.disable_pipes():
    doc_vectors = np.array([nlp(text).vector for text in data.Message])

print(f"{data.head(5)}\n\ndoc_vectors shape: {doc_vectors.shape}")

In the following cell data is being split into training and test sets. 

After splitting the data, since y_train is a list, it will still keep indices of original y list in range of [0-5572] even though its size after being split is 3900. Since Stratified K-Fold takes (X_train,y_train) as sets to split, indices of y_train list have to be reset so there are no "missing" values due to difference in indices.

In [ ]:
from sklearn.model_selection import train_test_split

X=doc_vectors
y=data.Category
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=388,
                                                        stratify=y, shuffle=True)
y_train.reset_index(drop=True,inplace=True)

print(f"y list shape: {y.shape} \ny_train list shape: {y_train.shape}")

To choose the best model, 4 models will be tested. SVC, LogisticRegression, a collection of Decision Trees as RandomForestClassifier and ANN as MLPClassifier.

To ensure best parameters of models for this dataset are being used, hyperparameters were tuned with Tune-sklearn's TuneSearchCV.

Run the next cell to set up models. 

In [ ]:
# Importing models
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

svc = SVC(C= 10, gamma= 0.1, kernel= 'rbf').fit(X_train,y_train)

logreg = LogisticRegression(solver='liblinear',random_state=388).fit(X_train,y_train)

rfc = RandomForestClassifier(bootstrap=False, max_depth=None,min_samples_leaf=1,min_samples_split=2,n_estimators=200).fit(X_train,y_train)

mlp = MLPClassifier(activation = 'relu', solver = 'adam', learning_rate='adaptive', alpha = 0.0001,
                      hidden_layer_sizes=(100,100,100)).fit(X_train,y_train)

After setting up models, in the next cell they're  going to be evaluated.

First method of evaluation is model.score() method. Since that method isn't reliable enough, evaluation will also include Cross Validation with f1 micro and f1 macro methods of scoring and Stratified K-Fold. Due to dataset being imbalanced (87% - 13%), emphasis will be on f1 macro score, as well as Stratified K-Fold.

Difference between f1 micro and f1 macro scores is that f1 micro gives equal importance to each sample whereas f1 macro gives equal importance to each class, which makes f1 macro more insightful when dealing with imbalanced dataset.

Since in cross-validation there is no guarantee every fold will have balanced portion of each class, to counter its randomness, Stratified K-Fold is used. Stratified K-Fold is cross-validation in which each fold has same distribution of classes, so there won't be inaccuracy by getting "lucky" with chosen folds.

In [ ]:
# Model evaluation
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

models=[svc,logreg,rfc,mlp]

for model in models:

    print(f"------------Model: {model}------------")
    print(f"Model score: {model.score(X_test,y_test)*100:.3f}%")
    
    skf = StratifiedKFold(n_splits=10)
    skf_splits = skf.get_n_splits(X_train, y_train)
    
    skf_scores=[]
    
    
    for train_index, test_index in skf.split(X_train, y_train):
        X_train_fold , X_test_fold = X_train[train_index], X_train[test_index]
        y_train_fold, y_test_fold = y_train[train_index], y_train[test_index]
        model.fit(X_train_fold, y_train_fold) 
        skf_scores.append(model.score(X_test_fold,y_test_fold))
    
    cv_scores_macro = cross_val_score(model, X, y, cv=skf_splits, scoring='f1_macro')
    cv_scores_micro = cross_val_score(model, X, y, cv=skf_splits, scoring='f1_micro')
    
    print(f"Cross-validation f1-macro score: {np.mean(cv_scores_macro) * 100:.3f}%")
    print(f"Cross-validation f1-micro score: {np.mean(cv_scores_micro) * 100:.3f}%")
    print(f"Stratified k-fold score: {np.mean(skf_scores) * 100:.3f}%\n")

As we can see, SVC algorithm is the most accurate with MLPClassifier being a close 2nd best. Other than accuracy, SVC is much more efficient than any of other algorithms whereas RandomForestClassifier was the slowest with MLPClassifier being in the 2nd place.